# **Études de cooccurrences de mots**
Auteur : PM Nugues

Une petite analyse des cooccurrences des racines *priv* et *publi[ck]* dans les débats du riksdag au XVI<sup>e</sup>, XVII<sup>e</sup> et XVIII<sup>e</sup> siècles. On peut appliquer ce programme à n'importe quelle séquence de caractères. Il suffit de changer quelques paramètres.

# Préliminaires

Les modules nécessaires

In [ ]:
import operator
import regex as re
import sys
from math import log, sqrt
from bs4 import BeautifulSoup
from tqdm import tqdm
from os import listdir

# Lecture des fichers XML et extraction des textes

## Liste des fichiers

On extrait la liste des fichers de débats. Les fichiers sont rassemblés pas décénnie.

In [ ]:
dossiers = ['1520-40', '1540-60', '1560-80', '1580-1600','1600-1620', '1620-40', '1640-60', '1660-80', '1680-1700',
           '1700-1720', '1720-40']
dossiers = ['Riksdagstryck/' + dossier + '/' for dossier in dossiers]
dossiers

In [ ]:
fichiers_par_décénnie = [listdir(dossier) for dossier in dossiers]
fichiers_par_décénnie2 = []
fichiers_tous = []
for i in range(len(dossiers)):
    fichiers_par_décénnie2 += [[dossiers[i] + fichiers for fichiers in fichiers_par_décénnie[i]]]
    fichiers_tous += [dossiers[i] + fichiers for fichiers in fichiers_par_décénnie[i]]
fichiers_par_décénnie2

## Lecture des fichiers

On fait l'analyse syntaxique du code XML de chaque fichier

In [ ]:
soups = []
for fichiers in fichiers_par_décénnie2:
    mini_soup = []
    for fichier in fichiers:
        print(fichier)
        with open(fichier, 'r', encoding='utf8') as infile:
            corpus_xml = infile.read()[1:]
        soup = BeautifulSoup(corpus_xml, 'xml')
        mini_soup += soup
    soups += [mini_soup]

On vérifie le nombre de décénnies

In [ ]:
len(soups)

On concatène les lignes de chaque fichier de sorte qu'il n'y ait qu'une ligne de texte par fichier

In [ ]:
textes = []
cnt = 0
print('Nom du fichier et nombre de lignes')
for mini_soups in soups:
    textes_par_décénnie = []
    for soup in mini_soups:
        lines = soup.find_all('line')
        print(fichiers_tous[cnt], ':', len(lines))
        cnt += 1
        texte = ''
        for line in lines:
            texte = texte + line.get_text() + '\n'
        #textes_par_décénnie += texte
        textes_par_décénnie += [texte]
    textes += [textes_par_décénnie]

On vérifie le nombre de textes

In [ ]:
len(textes)

Nombre de textes par décénnie

In [ ]:
list(map(len, textes))

# Concordances

Les séquences de caractères à rechercher et le contexte de la concordance

On fixe le choix à *priv* et *publi[ck]*. On peut les remplacer par n'importe quelle séquence

In [ ]:
PRIVAT = False

In [ ]:
if PRIVAT:
    chaîne = 'privat'
else:
    chaîne = 'publi[kc]'
contexte = 25

In [ ]:
chaîne = re.sub(' ', '\\s+', chaîne)

In [ ]:
for i, décénnie in enumerate(textes):
    print('===DÉCÉNNIE===', i + 1)
    for i, texte in enumerate(décénnie):
        print(fichiers[i], ':')
        texte = re.sub('\s+', ' ', texte.lower())
        concordance = ('(.{{0,{width}}}{pattern}.{{0,{width}}})'
                       .format(pattern=chaîne, width=contexte))
        for match in re.finditer(concordance, texte):
            print('  ', match.group(1))

# Cooccurrences

## Mots vides

On définit une liste de mots vides. On ne l'utilise pas dans le reste du programme

In [ ]:
mots_vides = ['och', 'den', 'också', 'för']

## Fonctions de comptage des mots et des bigrammes

On définit les fonctions de comptage. Elles sont tirées de l'édition à venir de mon livre.

In [ ]:
def découpe_mots(texte):
    mots = re.findall("\p{L}+|\p{N}+", texte)
    return mots


"""def découpe_mots(texte):
    mots = re.split('[\s\-,;:!?.’\'«»()–...&‘’“”*—]+', texte)
    mots.remove('')
    return mots"""


def compte_mots(mots):
    c_mots = {}
    for mot in mots:
        c_mots[mot] = c_mots.get(mot, 0) + 1
    return c_mots


def compte_bigrammes(mots):
    c_bigrammes = {}
    for i in range(len(mots) - 1):
        bigramme = (mots[i], mots[i + 1])
        c_bigrammes[bigramme] = c_bigrammes.get(bigramme, 0) + 1
    return c_bigrammes


def compte_bigrammes_fen(mots, fen=5):
    c_bigrammes_fen = {}
    for inx in range(1, fen + 1):
        for i in range(len(mots) - inx):
            bigramme = (mots[i], mots[i + inx])
            c_bigrammes_fen[bigramme] = c_bigrammes_fen.get(bigramme, 0) + 1
    return c_bigrammes_fen

On concatène les textes par décénnie. Pour chaque décénnie, on aura donc une ligne de texte

In [ ]:
textes = [' '.join(décénnie) for décénnie in textes]

## Les mots

On compte les mots

In [ ]:
c_mots_ordonnés_par_corpus = []
c_mots_par_corpus = []
textes_découpés = []
total_mots_par_corpus = []
for i, texte in enumerate(textes):
    #print(fichiers[i], ':')
    print('===DÉCÉNNIE===', i + 1)
    texte = texte.lower()
    texte_découpé = découpe_mots(texte)
    total_mots_par_corpus += [len(texte_découpé)]
    textes_découpés += [texte_découpé]
    # print(mots)
    c_mots = compte_mots(texte_découpé)
    c_mots_par_corpus += [c_mots]
    c_mots_ordonnés = sorted(c_mots.items(), key=operator.itemgetter(1), reverse=True)
    c_mots_ordonnés_par_corpus += [c_mots_ordonnés]
    print('  ', c_mots_ordonnés[:10])
print('Nombre total de mots:', total_mots_par_corpus)

On filtre les mots qui contiennent la séquence de caractères

In [ ]:
occurrences_chaîne = []
for i, c_mots_ordonnés in enumerate(c_mots_ordonnés_par_corpus):
    print('===DÉCÉNNIE===', i + 1)
    #print(fichiers[i], ':')
    occurrences_chaîne_1 = list(filter(lambda x: re.search(chaîne, x[0]), 
                      c_mots_ordonnés))
    occurrences_chaîne += [occurrences_chaîne_1]
    print(' ', list(filter(lambda x: re.search(chaîne, x[0]), 
                      c_mots_ordonnés)))

### Fréquence absolue de la séquence

In [ ]:
occurrences_chaîne

### Fréquence relative de la séquence par décénnie

In [ ]:
for i in range(len(total_mots_par_corpus)):
    print(sum([x[1] for x in occurrences_chaîne[i]])/total_mots_par_corpus[i])

## Les bigrammes

On compte les bigrammes et on les ordonne par fréquence

In [ ]:
c_bigrammes_ordonnés_par_corpus = []
c_bigrammes_par_corpus = []
for i, texte_découpé in enumerate(textes_découpés):
    print('===DÉCÉNNIE===', i + 1)
    # print(fichiers[i], ':')
    c_bigrammes = compte_bigrammes(texte_découpé)
    c_bigrammes_par_corpus += [c_bigrammes]
    c_bigrammes_ordonnés = sorted(c_bigrammes.items(), 
                                  key=operator.itemgetter(1), reverse=True)
    c_bigrammes_ordonnés_par_corpus += [c_bigrammes_ordonnés]
    print('  ', c_bigrammes_ordonnés[:15])

Les bigrammes qui contienne la séquence de caractères

In [ ]:
for i, c_bigrammes_ordonnés in enumerate(c_bigrammes_ordonnés_par_corpus):
    # print(fichiers[i], ':')
    print('===DÉCÉNNIE===', i + 1)
    print(' ', list(filter(lambda x: re.search(chaîne, x[0][0]) or 
                           re.search(chaîne, x[0][1]), 
                      c_bigrammes_ordonnés)))

Les bigrammes avec une fenêtre. On considère maintenant le contexte à droite et à gauche du mot

Les bigrammes les plus fréquents

In [ ]:
c_bigrammes_fen_ordonnés_par_corpus = []
for i, texte_découpé in enumerate(textes_découpés):
    # print(fichiers[i], ':')
    print('===DÉCÉNNIE===', i + 1)
    c_bigrammes_fen = compte_bigrammes_fen(texte_découpé)
    c_bigrammes_fen_ordonnés = sorted(c_bigrammes_fen.items(), 
                              key=operator.itemgetter(1), reverse=True)
    c_bigrammes_fen_ordonnés_par_corpus += [c_bigrammes_fen_ordonnés]
    print(c_bigrammes_fen_ordonnés[:10])

Les bgrammes avec la séquence de caractères recherchée

In [ ]:
for i, c_bigrammes_fen_ordonnés in enumerate(c_bigrammes_fen_ordonnés_par_corpus):
    #print(fichiers[i], ':')
    print('===DÉCÉNNIE===', i + 1)
    print(list(filter(lambda x: re.search(chaîne, x[0][0]) or 
                      re.search(chaîne, x[0][1]), 
                      c_bigrammes_fen_ordonnés)))

# Mesures de cooccurrence

On applique maintenant trois mesures de cooccurrence:
* l'information mutuelle de Fano
* les t-scores
* le rapport de vraisemblance

## Information mutuelle

Définition de l'information mutuelle

In [ ]:
def info_mutuelle(c_mots, c_bigrammes, taille):
    c_info_mutuelle = {}
    for bigramme in c_bigrammes.keys():
        c_info_mutuelle[bigramme] = log(taille * c_bigrammes[bigramme] / 
                                        (c_mots[bigramme[0]] * c_mots[bigramme[1]]), 2.0)
    return c_info_mutuelle

On ordonne les bigrammes par valeur d'information mutuelle décroissante

In [ ]:
c_info_mutuelle_ordonnée_par_corpus = []
c_info_mutuelle_par_corpus = []
for i, texte_découpé, c_mots, c_bigrammes in zip(range(len(textes)), 
                                  textes_découpés,
                                  c_mots_par_corpus, 
                                  c_bigrammes_par_corpus):
    #print(fichiers[i])
    print('===DÉCÉNNIE===', i + 1)
    c_info_mutuelle = info_mutuelle(c_mots, c_bigrammes, len(texte_découpé))
    c_info_mutuelle_par_corpus += [c_info_mutuelle]
    c_info_mutuelle_ordonnée = sorted(c_info_mutuelle.items(), 
                                      key=operator.itemgetter(1), reverse=True)
    print('  ', c_info_mutuelle_ordonnée[:10])
    c_info_mutuelle_ordonnée_par_corpus += [c_info_mutuelle_ordonnée]

L'information mutuelle des mots plus fréquents avec un seuil. Le seuil est assez haut pour le pas avoir trop d'affichages

In [ ]:
seuil = 200
for i, c_mots, c_bigrammes, c_info_mutuelle, c_info_mutuelle_ordonnée in zip(
    range(len(textes)),
    c_mots_par_corpus,
    c_bigrammes_par_corpus,
    c_info_mutuelle_par_corpus,
    c_info_mutuelle_ordonnée_par_corpus):
    #print(fichiers[i])
    print('===DÉCÉNNIE===', i + 1)
    for bigramme_im in c_info_mutuelle_ordonnée:
        if c_bigrammes[bigramme_im[0]] >= seuil:
            #mots_du_bigramme = bigramme_im[0].split()
            print('  ', c_info_mutuelle[bigramme_im[0]], "\t", 
                  bigramme_im[0], "\t", c_bigrammes[bigramme_im[0]], "\t",
                  c_mots[bigramme_im[0][0]], "\t", c_mots[bigramme_im[0][1]])

Les bigrammes avec la séquence qu'on recherche ordonnés par information mutuelle 

In [ ]:
for i, corpus in enumerate(c_info_mutuelle_ordonnée_par_corpus):
    print('===DÉCÉNNIE===', i + 1)
    print(list(filter(lambda x: re.search(chaîne, x[0][0]) or re.search(chaîne, x[0][1]), corpus)))

## t-scores

Définition des t-scores

In [ ]:
def t_scores(words, freq_unigrams, freq_bigrams):
    ts = {}
    for bigram in freq_bigrams:
        ts[bigram] = ((freq_bigrams[bigram] -
                      freq_unigrams[bigram[0]] *
                      freq_unigrams[bigram[1]] /
                      len(words)) /
                      sqrt(freq_bigrams[bigram]))
    return ts

On calcule les t-scores et on affiche les plus hauts

In [ ]:
c_t_scores_ordonnée_par_corpus = []
c_t_scores_par_corpus = []
for i, texte_découpé, c_mots, c_bigrammes in zip(range(len(textes)), 
                                  textes_découpés,
                                  c_mots_par_corpus, 
                                  c_bigrammes_par_corpus):
    # print(fichiers[i])
    print('===DÉCÉNNIE===', i + 1)
    c_t_scores = t_scores(texte_découpé, c_mots, c_bigrammes)
    c_t_scores_par_corpus += [c_t_scores]
    c_t_scores_ordonnée = sorted(c_t_scores.items(), 
                                      key=operator.itemgetter(1), reverse=True)
    print('  ', c_t_scores_ordonnée[:10])
    c_t_scores_ordonnée_par_corpus += [c_t_scores_ordonnée]

Les bigrammes avec la séquence qu'on recherche ordonnés par t-scores 

In [ ]:
for i, corpus in enumerate(c_t_scores_ordonnée_par_corpus):
    print('===DÉCÉNNIE===', i + 1)
    print(list(filter(lambda x: re.search(chaîne, x[0][0]) or re.search(chaîne, x[0][1]), corpus)))

## Log du rapport de vraisemblance (log-likelihood-ration)

Définition du rapport de vraisemblance

In [ ]:
def likelihood_ratio(words, freq_unigrams, freq_bigrams):
    lr = {}
    for bigram in freq_bigrams:
        p = freq_unigrams[bigram[1]] / len(words)
        p1 = freq_bigrams[bigram] / freq_unigrams[bigram[0]]
        p2 = ((freq_unigrams[bigram[1]] - freq_bigrams[bigram])
              / (len(words) - freq_unigrams[bigram[0]]))
        if p1 != 1.0 and p2 != 0.0:
            lr[bigram] = 2.0 * (
                log_f(freq_bigrams[bigram],
                      freq_unigrams[bigram[0]], p1) +
                log_f(freq_unigrams[bigram[1]] -
                      freq_bigrams[bigram],
                      len(words) - freq_unigrams[bigram[0]], p2) -
                log_f(freq_bigrams[bigram],
                      freq_unigrams[bigram[0]], p) -
                log_f(freq_unigrams[bigram[1]] -
                      freq_bigrams[bigram],
                      len(words) - freq_unigrams[bigram[0]], p))
    return lr

In [ ]:
def log_f(k, N, p):
    return k * log(p) + (N - k) * log(1 - p)

On le calcule

In [ ]:
c_lr_ordonné_par_corpus = []
c_lr_par_corpus = []
for i, texte_découpé, c_mots, c_bigrammes in zip(range(len(textes)), 
                                  textes_découpés,
                                  c_mots_par_corpus, 
                                  c_bigrammes_par_corpus):
    #print(fichiers[i])
    print('===DÉCÉNNIE===', i + 1)
    c_lr = likelihood_ratio(texte_découpé, c_mots, c_bigrammes)
    c_lr_par_corpus += [c_lr]
    c_lr_ordonné = sorted(c_lr.items(), 
                                      key=operator.itemgetter(1), reverse=True)
    print('  ', c_lr_ordonné[:10])
    c_lr_ordonné_par_corpus += [c_lr_ordonné]

Les bigrammes avec la séquence qu'on recherche ordonnés par rapports de vraisemblance

In [ ]:
for i, corpus in enumerate(c_lr_ordonné_par_corpus):
    print('===DÉCÉNNIE===', i + 1)
    print(list(filter(lambda x: re.search(chaîne, x[0][0]) or re.search(chaîne, x[0][1]), corpus)))